Cílem supercvičení je vyzkoušet si:
* jak dostat data do Hadoopu
* různé způsoby uložení dat v Hive (partitioning, formát, komprese)
* SQL dotazování nad velkými daty

## Nahrání dat do Hadoopu
Budeme pracovat se záznamy o průměrných teplotách na meteorologických stanicích v USA. Soubor s daty je uložen na lokálním FS:  
`/home/pascepet/fel_bigdata/data/teplota-usa.zip`  
Potřebujeme tato data dostat na Hadoop (tj. na HDFS). Tedy:  
* zkopírujte data do svého uživatelského adresáře (doporučuji si na to vytvořit podadresář);  
  `mkdir`  
  `cp`
* rozbalte;  
  `unzip`
* prohlédněte si několik řádků rozbalených dat a zjistěte počet řádků celkem (a zkuste si odpovědět na otázku, proč je dobré to udělat);  
  `head`  
  `wc -l`  
  *Chceme se podívat, jak data vypadají; počet řádků se hodí pro pozdější kontrolu.*
* vytvořte na HDFS ve svém uživatelském adresáři vhodný podadresář (stejná otázka jako výše &ndash; proč?);  
  `hdfs dfs -mkdir`  
  *Vytváříme samostatný podadresář, protože tabulka=adresář.*
* rozbalené soubory nakopírujte na HDFS do nově vytvořeného adresáře.  
  `hdfs dfs -put`

## Spuštění Hive konzoly
Spustíme Hive z příkazové řádky.

`beeline -u "jdbc:hive2://hador-c1.ics.muni.cz:10000/default;principal=hive/hador-c1.ics.muni.cz@ICS.MUNI.CZ"`

Pokud jste dosud nezaložili svou databázi Hive, vytvořte ji (jméno databáze zadejte jako své uživatelské jméno) a přepněte se do ní. Příkazy Hive je třeba ukončovat středníkem!  
`show databases;`  
`create database database_name;`  
`use database_name;`

## 1. Vstupní data jako externí tabulka
1.1. Ze souborů nahraných v předchozím kroku na HDFS vytvořte externí tabulku *teplota_ext*. (Externí tabulka používá strukturu dat tak, jak jsou, v tomto kroku se žádné změny formátu apod. neprovádějí.)
  * CSV (text) formát
  * oddělovač polí je znak ","
  * oddělovač záznamů je znak ukončení řádky
  * první řádek obsahuje hlavičky a přeskočí se
  * soubor obsahuje tato pole:

| Pole        | Typ       | Popis                                      |
|-------------|-----------|--------------------------------------------|
| stanice     | string    | kód stanice                                |
| mesic       | int       | číslo měsíce                               |
| den         | int       | číslo dne v měsíci                         |
| hodina      | int       | číslo hodiny (1&ndash;24)                  |
| teplota     | int       | průměrná teplota, round(10&times;&deg;F)   |
| flag        | string    | kód úplnosti dat                           |
| latitude    | double    | GPS zeměpisná šířka (záporné = jižní)      |
| longitude   | double    | GPS zeměpisná délka (záporné = západní)    |
| vyska       | double    | nadmořská výška v metrech                  |
| stat        | string    | kód státu (vč. zámořských území)           |
| nazev       | string    | název stanice                              |

```
create external table teplota_ext
( 
stanice string,
mesic int,
den int,
hodina int,
teplota int,
flag string,
latitude double,
longitude double,
vyska double,
stat string,
nazev string
)
row format delimited
fields terminated by ','
lines terminated by '\n'
stored as textfile
location 'cesta_k_adresari_s_daty_na_hdfs'
tblproperties ("skip.header.line.count"="1");
```

1.2. S externí tabulkou lze provádět SQL dotazy. Udělejte kontrolu:
  * výpis několika řádků tabulky *teplota_ext* a porovnání se vstupními daty;  
    `select * from teplota_ext limit 10;`
  * celkový počet řádků a porovnání se vstupními daty (neměl by být přesně stejný &ndash; proč?);  
    `select count(*) from teplota_ext;`  
    *Počet není stejný, protože se přeskakují úvodní řádky s hlavičkami sloupců.*
  * počet řádků s hodnotami NULL u pole *teplota* (měl by být jen malá část z celkového počtu řádků).  
    `select count(*) from teplota_ext where teplota is null;`

## 2. Převod do optimalizované tabulky

2.1. Vytvořte si prázdnou interní (managed) tabulku *teplota*, ve které budou data uložena ve vhodnějším formátu a zkomprimovaná:
  * formát parquet
  * bez partitioningu
  * komprese snappy (je nutno zadat velkými písmeny SNAPPY)
  * teplotu budeme v této tabulce ukládat ve stupních Celsia, bude to tedy desetinné číslo

```
create table teplota
( 
stanice string,
mesic int,
den int,
hodina int,
teplota double,
flag string,
latitude double,
longitude double,
vyska double,
stat string,
nazev string
)
stored as parquet
tblproperties("parquet.compress"="SNAPPY");
```

2.2. Vložte do tabulky *teplota* data z tabulky *teplota_ext*:
  * přepočtěte správně teplotu z 10&times;&deg;F na &deg;C;
  * ostatní pole převeďte beze změny.

```
insert overwrite table teplota
select
stanice, mesic, den, hodina, (teplota/10-32)*5/9,
flag, latitude, longitude, vyska, stat, nazev
from teplota_ext;
```

2.3. Udělejte kontrolu tabulky *teplota*:
  * Vypište si několik řádků.  
    `select * from teplota limit 10;`
  * Zjistěte počet záznamů v tabulce *teplota* a porovnejte s počtem záznamů v tabulce *teplota_ext*.  
    `select count(*) from teplota;`

2.4. Tabulka *teplota* je interní, a tedy jejím vlastníkem je Hive.
  * Najděte ji na HDFS pod `/user/hive/warehouse/jmeno_vasi_databaze.db` a zjistěte její velikost (počet MB).
  * Porovnejte velikost s velikostí externí tabulky (vámi nahraná data na HDFS, viz výše).

2.5. Zrušte v Hive externí tabulku *teplota_ext* (příkaz DROP TABLE). Prověřte, že tabulka sice již ve vaší databázi není, ale data na HDFS stále jsou.  
`drop table teplota_ext;`

## 3. Tabulka s partitions
3.1. Vytvořte si prázdnou interní (managed) tabulku *teplota_part*, která bude stejná jako tabulka *teplota* (tj. pole, jejich typy, formát, komprese), ale bude mít navíc partitioning podle měsíce. (Pozor na pořadí polí!)  
```
create table teplota_part
( 
stanice string,
den int,
hodina int,
teplota double,
flag string,
latitude double,
longitude double,
vyska double,
stat string,
nazev string
)
partitioned by (mesic int)
stored as parquet
tblproperties("parquet.compress"="SNAPPY");
```

3.2. Do tabulky *teplota_part* zkopírujte data z tabulky *teplota*, při kopírování vytvořte dynamický partitioning podle měsíce. Dynamický partitioning je potřeba předem povolit pomocí příkazů:  
```
set hive.exec.dynamic.partition=true;
set hive.exec.dynamic.partition.mode=nonstrict;

insert overwrite table teplota_part
partition (mesic)
select stanice, den, hodina, teplota,
flag, latitude, longitude, vyska, stat, nazev, mesic from teplota;
```

3.3. Najděte tabulku *teplota_part* na HDFS pod `/user/hive/warehouse/jmeno_vasi_databaze.db` a prohlédněte si, jak je partitioning realizován.  
*Každá partition je samostatný adresář.*

## 4. Dotazování nad Hive
Budeme pracovat s tabulkou *teplota*.

4.1. Kolik unikátních stanic je v datech? *(457)*  
`select count(distinct stanice) from teplota;`

4.2. Která stanice je nejsevernější? *(USW00027502, BARROW POST ROGERS AP)*   
`select * from teplota order by latitude desc limit 1;`

4.3. Který stát má nejvíc unikátních stanic? *(TX)*  
`select s.stat as stat, count(*) as pocet
from (select distinct stanice, stat from teplota) as s
group by s.stat
order by pocet desc
limit 1;`

4.4. Kolik hodinových údajů celkem je na některé ze stanic v severní Dakotě (ND) nižších než &minus;10&deg;C? *(8&nbsp;446)*  
`select sum(case when teplota<-10 then 1 else 0 end) as pocet
from teplota
where stat='ND';`

4.5. Který stát má nejvyšší celkovou průměrnou teplotu na svých stanicích za letní měsíce (6, 7, 8)? *(MH, 28.1)*  
`select stat, avg(teplota) as tepl_prum
from teplota
where mesic in (6, 7, 8)
group by stat
order by tepl_prum desc
limit 1;`

4.6. Které všechny státy mají rozdíl zeměpisných šířek (longitude) mezi svou nejzápadnější a nejvýchodnější stanicí větší než 8 stupňů? *(AK, FM, MT, TX)*  
`select stat, max(longitude)-min(longitude) as long_rozdil
from teplota
group by stat
having max(longitude)-min(longitude)>8;`

4.7. Pro každou stanici s nadmořskou výškou nad 1&nbsp;500&nbsp;metrů zjistěte rozdíl mezi celkovou průměrnou teplotou stanice (za celý rok) a celkovou průměrnou teplotou státu, kam stanice patří.

| stanice     | rozdil             |
|:------------|-------------------:|
| USW00003103 | -9.732538702083291 |
| USW00023225 | -4.890214469033042 |
| ...         | ...                |

*(celkem 20 stanic)*

`select p.stanice, p.stan_prum-s.stat_prum as rozdil from
  (select stanice, stat, avg(vyska) as vyska, avg(teplota) as stan_prum
   from teplota where vyska>1500 group by stanice, stat) as p
join
  (select stat, avg(teplota) as stat_prum from teplota group by stat) as s
  on p.stat=s.stat;`